In [1]:
import numpy as np 
import pandas as pd 
import xarray as xr 
import netCDF4 

In [3]:
# ################ Settings. Change with respect to your system ###################
coarse_lat = np.arange(-20., 20.1, 2.0)
coarse_long = np.arange(0., 359.9, 2.0)
coarse_long

array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
        22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
        44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
        66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
        88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
       110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
       132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
       154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
       176., 178., 180., 182., 184., 186., 188., 190., 192., 194., 196.,
       198., 200., 202., 204., 206., 208., 210., 212., 214., 216., 218.,
       220., 222., 224., 226., 228., 230., 232., 234., 236., 238., 240.,
       242., 244., 246., 248., 250., 252., 254., 256., 258., 260., 262.,
       264., 266., 268., 270., 272., 274., 276., 278., 280., 282., 284.,
       286., 288., 290., 292., 294., 296., 298., 30

In [2]:
from pathlib import Path
import os.path

import mjoindices.olr_handling as olr
import mjoindices.omi.omi_calculator as omi
import mjoindices.empirical_orthogonal_functions as eof
import mjoindices.principal_components as pc
import mjoindices.evaluation_tools
import numpy as np
import xarray as xr 
import pandas as pd 

class OLRData:
    """
    This class serves as a container for spatially distributed and temporally resolved OLR data.

    A filled object of this class has to be provided by the user in order to start the OMI calculation.

    :param olr: The OLR data as a 3-dim array. The three dimensions correspond to time, latitude, and longitude, in this
        order.
    :param time: The temporal grid as 1-dim array of :class:`numpy.datetime64` dates.
    :param lat: The latitude grid as 1-dim array.
    :param long: The longitude grid as 1-dim array.
    """

    def __init__(self, olr: np.ndarray, time: np.ndarray, lat: np.ndarray, long: np.ndarray) -> None:
        """
        Initialization of basic variables.
        """
        if olr.shape[0] != time.size:
            raise ValueError('Length of time grid does not fit to first dimension of OLR data cube')
        if olr.shape[1] != lat.size:
            raise ValueError('Length of lat grid does not fit to second dimension of OLR data cube')
        if olr.shape[2] != long.size:
            raise ValueError('Length of long grid does not fit to third dimension of OLR data cube')
        self._olr = olr.copy()
        self._time = time.copy()
        self._lat = lat.copy()
        self._long = long.copy()

    @property
    def olr(self):
        """
        The OLR data as a 3-dim array. The three dimensions correspond to time, latitude, and longitude, in this
        order.
        """
        return self._olr

    @property
    def time(self):
        """
        The temporal grid as 1-dim array of :class:`numpy.datetime64` dates.
        """
        return self._time

    @property
    def lat(self):
        """
        The latitude grid as 1-dim array.
        """
        return self._lat

    @property
    def long(self):
        """
        The longitude grid as 1-dim array.
        """
        return self._long

    def __eq__(self, other: "OLRData") -> bool:
        """
        Override the default Equals behavior
        """
        return (np.all(self.lat == other.lat)
                and np.all(self.long == other.long)
                and np.all(self.time == other.time)
                and np.all(self.olr == other.olr))

    def close(self, other: "OLRData") -> bool:
        """
         Checks equality of two :class:`OLRData` objects, but allows for numerical tolerances.

        :param other: The object to compare with.

        :return: Equality of all members considering the default tolerances of :func:`numpy.allclose`
        """
        return (np.allclose(self.lat, other.lat)
                and np.allclose(self.long, other.long)
                and np.allclose(self.time.astype("float"), other.time.astype("float"))  # allclose does not work with datetime64
                and np.allclose(self.olr, other.olr))


    def get_olr_for_date(self, date: np.datetime64) -> np.ndarray:
        """
        Returns the spatially distributed OLR map for a particular date.

        :param date: The date, which hat to be exactly matched by one of the dates in the OLR time grid.

        :return: The excerpt of the OLR data as a 2-dim array. The two dimensions correspond to
            latitude, and longitude, in this order. Returns None if the date is not contained in the OLR time series.
        """
        cand = self.time == date
        if not np.all(cand == False):  # noqa: E712
            return np.squeeze(self.olr[cand, :, :])
        else:
            return None


    def extract_olr_matrix_for_doy_range(self, center_doy: int, window_length: int = 0,
                                         strict_leap_year_treatment: bool = False) -> np.ndarray:
        """
        Extracts the OLR data, which belongs to all DOYs around one center (center_doy +/- windowlength).

        Keep in mind that the OLR time series might span several years. In this case the center DOY is found more than
        once and the respective window in considered for each year.
        Example: 3 full years of data, centerdoy = 20, and window_length = 4 results in 3*(2*4+1) = 27 entries in the
        time axis

        :param center_doy: The center DOY of the window.
        :param window_length: The window length in DOYs on both sides of the center DOY. Hence, if the window is fully
            covered by the data, one gets 2*window_length + 1 entries per year in the result.
        :param strict_leap_year_treatment: see description in :meth:`mjoindices.tools.find_doy_ranges_in_dates`.

        :return: The excerpt of the OLR data as a 3-dim array. The three dimensions correspond to
            time, latitude, and longitude, in this order.
        """
        inds, doys = tools.find_doy_ranges_in_dates(self.time, center_doy, window_length=window_length,
                                                    strict_leap_year_treatment=strict_leap_year_treatment)
        return self.olr[inds, :, :]


    def save_to_npzfile(self, filename: Path) -> None:
        """
        Saves the data arrays contained in the OLRData object to a numpy file.

        :param filename: The full filename.
        """
        np.savez(filename, olr=self.olr, time=self.time, lat=self.lat, long=self.long)

# ################ Settings. Change with respect to your system ###################

# # Choose a spatial grid, on which the values are computed.
olr_data_filename = '/pscratch/sd/l/linyaoly/ERA5/reanalysis/ERA5.olr.day.1978to2022.nc'
eofnpzfile = '/pscratch/sd/l/linyaoly/ERA5/EOF/EOFs1979to2014.npz'
pctxtfile = '/pscratch/sd/l/linyaoly/ERA5/EOF/ERA5_OMI_daily.nc'

# Load the OLR data.
# This is the first place to insert your own OLR data, if you want to compute OMI for a different dataset.
ds = xr.open_dataset(olr_data_filename)
time = ds['time']
lon  = ds['lon']
lat  = ds['lat']
# convert unit to W/m2
olr  = - ds['olr'] / 3600

if np.sum(olr.isnull()).values:
    print('missing values in OLR data!')
    exit()

time = np.asarray(time)
lon  = np.asarray(lon)
lat  = np.asarray(lat)
olr  = np.asarray(olr)

raw_olr = OLRData(olr, time, lat, lon)

raw_olr

In [17]:
olr_data_filename = '/pscratch/sd/l/linyaoly/ERA5/reanalysis/ERA5.olr.day.1978to2022.nc'

ds = xr.open_dataset(olr_data_filename)

time = ds['time']
lon  = ds['lon']
lat  = ds['lat']
olr  = ds['olr']

In [24]:
# check if there is missing values
np.sum(olr.isnull()).values 

array(0)

In [25]:
olr  = - ds['olr'] / 3600
olr 

<xarray.DataArray 'olr' (time: 16222, lat: 91, lon: 180)>
array([[[155.5425  , 155.5425  , 155.5425  , ..., 155.5425  ,
         155.5425  , 155.5425  ],
        [155.25471 , 154.94708 , 154.63013 , ..., 157.36722 ,
         156.63652 , 155.87932 ],
        [159.1323  , 158.48648 , 158.16423 , ..., 161.19572 ,
         160.26878 , 159.79535 ],
        ...,
        [188.8227  , 188.57603 , 188.3201  , ..., 190.03212 ,
         189.59317 , 189.19005 ],
        [186.41316 , 186.31636 , 186.19568 , ..., 186.32431 ,
         186.36012 , 186.38795 ],
        [172.24757 , 172.24757 , 172.24757 , ..., 172.24757 ,
         172.24757 , 172.24757 ]],

       [[155.00941 , 155.00941 , 155.00941 , ..., 155.00941 ,
         155.00941 , 155.00941 ],
        [157.23064 , 156.82484 , 156.44426 , ..., 159.1482  ,
         158.58594 , 157.97726 ],
        [162.04576 , 162.63058 , 163.35861 , ..., 160.38945 ,
         160.70639 , 161.33365 ],
...
        [130.68614 , 130.42085 , 130.17073 , ..., 132.0985  ,
         131.5679  , 131.08913 ],
        [128.14442 , 128.1002  , 127.94481 , ..., 128.26064 ,
         128.18105 , 128.11662 ],
        [126.27097 , 126.27097 , 126.27097 , ..., 126.27097 ,
         126.27097 , 126.27097 ]],

       [[222.59868 , 222.59868 , 222.59868 , ..., 222.59868 ,
         222.59868 , 222.59868 ],
        [224.72227 , 224.99387 , 225.28821 , ..., 223.50067 ,
         223.94029 , 224.36728 ],
        [230.00024 , 230.25797 , 230.4904  , ..., 230.69884 ,
         230.42091 , 230.22131 ],
        ...,
        [129.77531 , 129.39001 , 128.99966 , ..., 131.61212 ,
         130.91858 , 130.33368 ],
        [128.0762  , 128.06737 , 127.99282 , ..., 128.33139 ,
         128.2     , 128.09135 ],
        [139.67818 , 139.67818 , 139.67818 , ..., 139.67818 ,
         139.67818 , 139.67818 ]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1978-01-01 1978-01-02 ... 2022-05-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0

In [26]:
ds = xr.open_dataset(olr_data_filename)
time = ds['time']
lon  = ds['lon']
lat  = ds['lat']
olr  = - ds['olr'] / 3600

if np.sum(olr.isnull()).values:
    print('missing values in OLR data!')
    exit()

time = np.asarray(time)
lon  = np.asarray(lon)
lat  = np.asarray(lat)
olr  = np.asarray(olr)
olr 

array([[[155.5425  , 155.5425  , 155.5425  , ..., 155.5425  ,
         155.5425  , 155.5425  ],
        [155.25471 , 154.94708 , 154.63013 , ..., 157.36722 ,
         156.63652 , 155.87932 ],
        [159.1323  , 158.48648 , 158.16423 , ..., 161.19572 ,
         160.26878 , 159.79535 ],
        ...,
        [188.8227  , 188.57603 , 188.3201  , ..., 190.03212 ,
         189.59317 , 189.19005 ],
        [186.41316 , 186.31636 , 186.19568 , ..., 186.32431 ,
         186.36012 , 186.38795 ],
        [172.24757 , 172.24757 , 172.24757 , ..., 172.24757 ,
         172.24757 , 172.24757 ]],

       [[155.00941 , 155.00941 , 155.00941 , ..., 155.00941 ,
         155.00941 , 155.00941 ],
        [157.23064 , 156.82484 , 156.44426 , ..., 159.1482  ,
         158.58594 , 157.97726 ],
        [162.04576 , 162.63058 , 163.35861 , ..., 160.38945 ,
         160.70639 , 161.33365 ],
        ...,
        [191.32774 , 191.0917  , 190.83177 , ..., 192.42976 ,
         192.0319  , 191.64998 ],
        [190

In [10]:
np.array(f.variables['time'][:], dtype='datetime64[D]')

array(['1970-01-01', '1970-01-02', '1970-01-03', ..., '2014-05-29',
       '2014-05-30', '2014-05-31'], dtype='datetime64[D]')

In [ ]:
days_since1978 = time 
temptime = []
for item in days

In [4]:
ds.variables['time']

<class 'netCDF4._netCDF4.Variable'>
int64 time(time)
    units: days since 1978-01-01 00:00:00
    calendar: proleptic_gregorian
unlimited dimensions: 
current shape = (16222,)
filling on, default _FillValue of -9223372036854775806 used

In [5]:
hours_since1800 = ds.variables['time'][:].data.copy()

temptime = []
for item in hours_since1800:
    delta = np.timedelta64(int(item / 24), 'D')
    day = np.datetime64('1800-01-01') + delta
    temptime.append(day)
time = np.array(temptime, dtype=np.datetime64)

time 
    

array(['1800-01-01', '1800-01-01', '1800-01-01', ..., '1801-11-07',
       '1801-11-07', '1801-11-07'], dtype='datetime64[D]')

In [8]:
time[0:50]

array(['1800-01-01', '1800-01-01', '1800-01-01', '1800-01-01',
       '1800-01-01', '1800-01-01', '1800-01-01', '1800-01-01',
       '1800-01-01', '1800-01-01', '1800-01-01', '1800-01-01',
       '1800-01-01', '1800-01-01', '1800-01-01', '1800-01-01',
       '1800-01-01', '1800-01-01', '1800-01-01', '1800-01-01',
       '1800-01-01', '1800-01-01', '1800-01-01', '1800-01-01',
       '1800-01-02', '1800-01-02', '1800-01-02', '1800-01-02',
       '1800-01-02', '1800-01-02', '1800-01-02', '1800-01-02',
       '1800-01-02', '1800-01-02', '1800-01-02', '1800-01-02',
       '1800-01-02', '1800-01-02', '1800-01-02', '1800-01-02',
       '1800-01-02', '1800-01-02', '1800-01-02', '1800-01-02',
       '1800-01-02', '1800-01-02', '1800-01-02', '1800-01-02',
       '1800-01-03', '1800-01-03'], dtype='datetime64[D]')

In [10]:
ds = xr.open_dataset(olr_data_filename)
ds 

<xarray.Dataset>
Dimensions:  (time: 16222, lon: 180, lat: 91)
Coordinates:
  * time     (time) datetime64[ns] 1978-01-01 1978-01-02 ... 2022-05-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
Data variables:
    olr      (time, lat, lon) float32 ...

In [11]:
ds = ds.sel(time=slice('1979-01-01','2012-12-31'), lat=slice(20, -20))
ds 


<xarray.Dataset>
Dimensions:  (time: 12419, lon: 180, lat: 21)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2012-12-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 20.0 18.0 16.0 14.0 12.0 ... -14.0 -16.0 -18.0 -20.0
Data variables:
    olr      (time, lat, lon) float32 ...

In [12]:
ds.olr 

<xarray.DataArray 'olr' (time: 12419, lat: 21, lon: 180)>
[46943820 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2012-12-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 20.0 18.0 16.0 14.0 12.0 ... -14.0 -16.0 -18.0 -20.0

In [15]:
ds['olr'] = - ds['olr'] / 3600
ds 

<xarray.Dataset>
Dimensions:  (time: 12419, lon: 180, lat: 21)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2012-12-31
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float32 20.0 18.0 16.0 14.0 12.0 ... -14.0 -16.0 -18.0 -20.0
Data variables:
    olr      (time, lat, lon) float32 284.5 262.8 266.4 ... 297.3 299.9 296.8